# Vessel Seg 全流程示例（ASOCA/RCF/Shape）

按顺序演示：环境检查 → 数据准备 → 形状特征 → 分段 → RCF 推理（PNG→NIfTI）。命令均在仓库根目录执行。


In [ ]:
import sys, os, json, subprocess, textwrap
from pathlib import Path
print("cwd:", Path.cwd())
print("Python:", sys.version)
print("vesselfm env?", os.environ.get("CONDA_DEFAULT_ENV"))

## 1. 形状特征与分段（以 Normal_1 为例）


In [ ]:
%%bash
# 提取中心线 + 极坐标截面特征
/home/chenyihao/miniconda3/envs/vesselfm/bin/python -m vessel_seg.shape extract \
  --seg ASOCA2020/Normal/Annotations_nii/Normal_1.nii.gz \
  --out outputs/Normal_1_features

In [ ]:
%%bash
# 按弧长 10mm 切段，记录相邻段夹角
/home/chenyihao/miniconda3/envs/vesselfm/bin/python scripts/split_branch_segments.py \
  --features outputs/Normal_1_features \
  --out outputs/Normal_1_segments \
  --segment-length 10 \
  --min-segment-length 2

In [ ]:
import json, pprint, pathlib
summary = json.loads(pathlib.Path('outputs/Normal_1_segments/segments_summary.json').read_text())
print('分支段数:', summary.get('segment_count'))
pprint.pp(summary['segments'][:5])

## 2. RCF 推理（PNG → NIfTI）
- 假设已有 RCF 训练好的权重 `results/RCF/ASOCA2020/checkpoint_epoch20.pth`。
- 若需重新切片可用 `scripts/prepare_rcf_asoca.py`，此处直接演示推理 + 拼成 NIfTI。


In [ ]:
%%bash
set -e
cd third_party/RCF-PyTorch
# 对 Normal_1 的 PNG 目录做推理；根据实际存放修改 --input-dir
python Ktest.py \
  --checkpoint results/RCF/ASOCA2020/checkpoint_epoch20.pth \
  --input-dir data/ASOCA2020_rcf/train/img \
  --save-dir results/RCF/ASOCA2020_predpng_Normal1

In [ ]:
%%bash
# 将预测 PNG 堆成与原 CTA 对齐的 NIfTI
python scripts/rcf_to_nifti.py \
  --rcf-dir third_party/RCF-PyTorch/results/RCF/ASOCA2020_predpng_Normal1 \
  --reference ASOCA2020/Normal/CTCA_nii/Normal_1.nii.gz \
  --output outputs/Normal_1_rcf_edges.nii.gz

### 可选：简化版推理（支持切换是否取反）
- 避免 Ktest 中的固定路径/参数，直接加载 checkpoint，逐切片推理 CTA 并生成 NIfTI。
- `invert=False` 时不再执行 `1 - fuse`，若发现预测与 GT 反相，可切换为 `True`。

In [ ]:
import sys, cv2, torch, nibabel as nib, numpy as np
from pathlib import Path
sys.path.insert(0, str(Path('third_party/RCF-PyTorch').resolve()))
from models import RCF

ckpt = Path('third_party/RCF-PyTorch/results/RCF/ASOCA2020/checkpoint_epoch20.pth')
ref_path = Path('ASOCA2020/Normal/CTCA_nii/Normal_1.nii.gz')
out_path = Path('outputs/Normal_1_rcf_edges_simplified.nii.gz')
invert = False  # 如需反相切到 True

img = nib.load(str(ref_path))
vol = img.get_fdata().astype(np.float32)
mean = np.array([104.00698793, 116.66876762, 122.67891434], dtype=np.float32)

model = RCF().cuda()
state = torch.load(str(ckpt), map_location='cuda', weights_only=False)
state_dict = state.get('state_dict', state)
model.load_state_dict(state_dict, strict=False)
model.eval()

edges = np.zeros_like(vol, dtype=np.float32)
with torch.no_grad():
    for z in range(vol.shape[0]):
        slice_arr = vol[z]
        vmax = float(np.max(slice_arr))
        slice_norm = cv2.convertScaleAbs(slice_arr, alpha=(255.0 / max(1e-3, vmax)))
        img_rgb = cv2.cvtColor(slice_norm, cv2.COLOR_GRAY2RGB).astype(np.float32)
        img_rgb = (img_rgb - mean).transpose(2,0,1)[None]
        inp = torch.from_numpy(img_rgb).cuda()
        fuse = torch.squeeze(model(inp)[-1]).detach().cpu().numpy().astype(np.float32)
        edges[z] = (1.0 - fuse) if invert else fuse

nib.save(nib.Nifti1Image(edges, img.affine, img.header), str(out_path))
out_path

## 3. 可选：重建 3D 网格或与 FGPM 融合
- 重建网格：`python -m vessel_seg.shape reconstruct --features outputs/Normal_1_features --output outputs/Normal_1.vtp`
- 在 FGPM 推理中使用边缘先验：`python -m vessel_seg.fgpm infer --edge-map outputs/Normal_1_rcf_edges.nii.gz ...`

以上 notebook 单元可按需修改路径/参数后依次运行。